The purpose of this code is to measure the flourescence intensity of cells, which represents an indicator of the glucose uptake of the cells, that have a knockdown of ADAMTS19 or TAX1BP1 against a SCR control.

Import Libraries

In [ ]:
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb, rgb2hsv
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk, local_maxima
from scipy.ndimage import binary_fill_holes
from skimage.feature import blob_log
from skimage.color import rgb2gray
from skimage.draw import circle_perimeter
from matplotlib.colors import LogNorm
import scipy.ndimage as ndi
import pandas as pd
import os
from cellpose import models, io, plot
from skimage import morphology, io
from IPython.display import clear_output
from scipy.ndimage import zoom
import time
model = models.Cellpose(model_type='cyto')

Define Sub Function

In [ ]:
target_shape = (1584, 1584)

def analyze_image(image_path, basename):
    """
    Analyzes an image to segment cells, calculate cell sizes, and mean fluorescence intensities (MFI).
    
    Args:
        image_path (str): Path to the image file to be analyzed.
        basename (str): Base name of the image file (used for labeling results).

    Returns:
        pd.DataFrame: A DataFrame containing the filename, cell numbers, cell sizes, and cell MFIs.
    """
    # Load the image using czifile and remove singleton dimensions
    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)
    green_channel = image_squeezed  # Assume green_channel contains the relevant data

    # Initialize lists to store results
    cell_nums = []
    cell_sizes = []
    cell_mfi = []

    # Display the green channel image
    plt.figure(figsize=(6, 6))  # Adjust the figure size as needed
    plt.imshow(green_channel)
    plt.axis('off')  # Turn off axis labels and ticks for a cleaner image

    # Save the figure with minimal white space
    plt.tight_layout(pad=0)
    plt.savefig('temp_img.png')  # Save the displayed image to a temporary file

    time.sleep(2)  # Pause to ensure the image is saved (may be unnecessary)

    # Read the saved image
    img = io.imread('temp_img.png')

    # Use the model to evaluate the image and obtain masks, flows, styles, and diameters
    masks, flows, styles, diams = model.eval(img, diameter=None)
    
    # Label the connected regions in the mask
    labeled_image = label(masks)

    # Iterate over each labeled region to calculate cell size and MFI
    for i, region in enumerate(regionprops(labeled_image)):
        cell_size = region.area  # Calculate the area of the cell

        if region.area < 1000:  # Filter out small regions (e.g., noise)
            continue

        # Create a binary mask for the current region (cell)
        mask = labeled_image == region.label
        
        # Resize the mask to match a target shape using zoom (may alter mask binary values)
        mask = zoom(mask, zoom=np.array(target_shape) / np.array(mask.shape), order=0)
        
        # Calculate the mean fluorescence intensity (MFI) for the region
        interior_mean = np.mean(green_channel[mask > 0])
        
        # Append results to lists
        cell_nums.append(i)
        cell_sizes.append(cell_size)
        cell_mfi.append(interior_mean)
       
    # Create a DataFrame with the results
    df = pd.DataFrame({
        "Filename": [basename] * len(cell_nums),  # Repeat basename for each cell
        "Cell": cell_nums,                        # List of cell indices
        "Cell_Size": cell_sizes,                  # List of cell sizes
        "cell_mfi": cell_mfi                      # List of mean fluorescence intensities
    })

    return df  # Return the DataFrame with analysis results

Define Main Function

In [ ]:
all_data = []
big_folder = r"test_images"

for well_image in os.listdir(big_folder):
    if well_image.lower().endswith(".czi"):  # Filter for CZI files
        well_image_path = os.path.join(big_folder, well_image)
        well_image_base_name = os.path.basename(well_image)[:-4]
        df = analyze_image(well_image_path, well_image_base_name)
        if df is not None:
            all_data.append(df)
        clear_output(wait=True)

combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a single CSV
combined_csv_path = os.path.join("GLUCOSE.xlsx")
combined_df.to_excel(combined_csv_path, index=False)